In [1]:
#@title Check if installation was successful

#from google.colab import files

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Tue Oct 31 08:02:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.120                Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN RTX               On  | 00000000:01:00.0  On |                  N/A |
| 41%   34C    P8              14W / 280W |   1301MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

/tmp/ipykernel_2198/3038417431.py:5: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.util


In [2]:
import random
import gymnasium as gym
env = gym.make('Hopper', render_mode='rgb_array')
env.reset()
n_frames = 120
height = 480
width = 480
frames = []
#import mujoco
model = env.model
data = env.data

mujoco.mj_step(model, data)
renderer = mujoco.Renderer(model, height, width)
mujoco.mj_resetDataKeyframe(model, data,0)

joint_names = ['thigh_joint', 'leg_joint', 'foot_joint']
joint_ranges = [model.joint(joint).range for joint in joint_names]
for i in range(n_frames):
  while data.time < i/30.0:   
    new_action = []
        
    # ob, reward, terminated, False, {}
    obs, reward, terminated, _, _ = env.step(new_action)
    #mujoco.mj_step(model, data)
  renderer.update_scene(data, "y")
  frame = renderer.render()
  frames.append(frame)
media.show_video(frames, fps=30)


/research/agent-playground/dm_mujoco/dm_venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Using the latest versioned environment `Hopper-v4` instead of the unversioned environment `Hopper`.
  logger.warn(
/research/agent-playground/dm_mujoco/dm_venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.model to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.model` for environment variables or `env.get_wrapper_attr('model')` that will search the reminding wrappers.
  logger.warn(
/research/agent-playground/dm_mujoco/dm_venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.data` for environment variables or `env.get_wrapper_attr('data')` that will search the reminding wrappers.
  logger.warn(


ValueError: Action dimension mismatch. Expected (3,), found (0,)

In [2]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
from stable_baselines3.common.evaluation import evaluate_policy

# Create your environment
env = gym.make('Hopper', render_mode = 'rgb_array')

# Wrap the environment to record a video every 100 episodes
video_folder = './videos/'
video_length = 500  # Length of recorded video

def make_record_env(env_id):
    env = DummyVecEnv([lambda: gym.make(env_id, render_mode = 'rgb_array')])
    env = VecVideoRecorder(env, video_folder,
                           record_video_trigger=lambda x: x % 100 == 0,  # Trigger recording every 100 episodes
                           video_length=video_length,
                           name_prefix="PPO-{}".format(env_id),)
    return env

record_env = make_record_env('Hopper')

# Initialize the PPO model
model = PPO("MlpPolicy", record_env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Close the environment
record_env.close()


/research/agent-playground/dm_mujoco/dm_venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Using the latest versioned environment `Hopper-v4` instead of the unversioned environment `Hopper`.
  logger.warn(


Using cuda device
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-0-to-step-500.mp4
Moviepy - Building video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-0-to-step-500.mp4.
Moviepy - Writing video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-0-to-step-500.mp4
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-600-to-step-1100.mp4
Moviepy - Building video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-600-to-step-1100.mp4.
Moviepy - Writing video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-600-to-step-1100.mp4



Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-600-to-step-1100.mp4
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1200-to-step-1700.mp4
Moviepy - Building video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1200-to-step-1700.mp4.
Moviepy - Writing video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1200-to-step-1700.mp4



Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1200-to-step-1700.mp4
-----------------------------
| time/              |      |
|    fps             | 50   |
|    iterations      | 1    |
|    time_elapsed    | 40   |
|    total_timesteps | 2048 |
-----------------------------
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1800-to-step-2300.mp4
Moviepy - Building video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1800-to-step-2300.mp4.
Moviepy - Writing video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-1800-to-step-2300.mp4



...

Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-6000-to-step-6500.mp4
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-6600-to-step-7100.mp4
Moviepy - Building video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-6600-to-step-7100.mp4.
Moviepy - Writing video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-6600-to-step-7100.mp4



Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-6600-to-step-7100.mp4
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-7200-to-step-7700.mp4
Moviepy - Building video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-7200-to-step-7700.mp4.
Moviepy - Writing video /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-7200-to-step-7700.mp4



Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-7200-to-step-7700.mp4
----------------------------------------
| time/                   |            |
|    fps                  | 49         |
|    iterations           | 4          |
|    time_elapsed         | 163        |
|    total_timesteps      | 8192       |
| train/                  |            |
|    approx_kl            | 0.01481018 |
|    clip_fraction        | 0.167      |
|    clip_range           | 0.2        |
|    entropy_loss         | -4.24      |
|    explained_variance   | 0.228      |
|    learning_rate        | 0.0003     |
|    loss                 | 86.9       |
|    n_updates            | 30         |
|    policy_gradient_loss | -0.025     |
|    std                  | 0.993      |
|    value_loss           | 160        |
----------------------------------------
Saving video to /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-7800-to-step-8300.mp4
Mo

Moviepy - Done !
Moviepy - video ready /research/agent-playground/dm_mujoco/videos/PPO-Hopper-step-7800-to-step-8300.mp4


KeyboardInterrupt: 

In [19]:
import random


n_frames = 120
height = 480
width = 480
frames = []
#import mujoco
xml_name = 'scene.xml'
model = mujoco.MjModel.from_xml_path(f"./mujoco_menagerie/franka_emika_panda/{xml_name}")
data = mujoco.MjData(model)
mujoco.mj_step(model, data)
renderer = mujoco.Renderer(model, height, width)
mujoco.mj_resetDataKeyframe(model, data,0)

joint_names = [model.joint(i).name for i in range(model.njnt)]

'''
for joint in joint_names :
    joint_range = model.joint(joint).range
    action = random.uniform(model.joint(joint).range[0], model.joint(joint).range[1])
    data.qpos[model.joint(joint).id] = action    
'''


# pose mocap to wrist_3
mujoco.mj_step(model, data)

for i in range(n_frames):
  while data.time < i/30.0:   
    #print(data.xpos[model.body('wrist_3_link').id])
    #print(data.qpos)
    for joint in joint_names :
        if len(joint) == 0 :
          continue
        joint_range = model.joint(joint).range
        action_target = random.uniform(model.joint(joint).range[0], model.joint(joint).range[1])
        action_delta = action_target - data.qpos[model.joint(joint).id]
        action = data.qpos[model.joint(joint).id] + action_delta * 0.01
        data.qpos[model.joint(joint).id] = action 
    mujoco.mj_step(model, data)

  renderer.update_scene(data, "y")
  frame = renderer.render()
  frames.append(frame)
media.show_video(frames, fps=30)

This browser does not support the video tag.


In [6]:
joint_names = [model.joint(i).name for i in range(model.njnt)]
print(joint_names)

['joint_torso', 'joint_shoulder', 'joint_bicep', 'joint_elbow', 'joint_forearm', 'joint_wrist', 'joint_gripper', 'joint_finger_right', 'joint_finger_left']


In [14]:
import random


n_frames = 120
height = 480
width = 480
frames = []
#import mujoco
xml_name = 'scene_paper_1.xml'
!cp ./scene_paper_1.xml ./mujoco_menagerie/universal_robots_ur5e/
!cp ./ur5e.xml ./mujoco_menagerie/universal_robots_ur5e/
model = mujoco.MjModel.from_xml_path(f"./mujoco_menagerie/universal_robots_ur5e/{xml_name}")
data = mujoco.MjData(model)
mujoco.mj_step(model, data)
renderer = mujoco.Renderer(model, height, width)
mujoco.mj_resetDataKeyframe(model, data,0)

# UR5E 관절 이름 및 초기 각도 값
initial_joint_angles = {
    'shoulder_pan_joint': 5.462,
    'shoulder_lift_joint': -1.887,
    'elbow_joint': 1.321,
    'wrist_1_joint': -1.013,
    'wrist_2_joint': -1.582,
    'wrist_3_joint': -4.824
}

joint_names = [model.joint(i).name for i in range(model.njnt)]

'''
for joint in joint_names :
    joint_range = model.joint(joint).range
    action = random.uniform(model.joint(joint).range[0], model.joint(joint).range[1])
    data.qpos[model.joint(joint).id] = action    
'''

for joint_name, angle in initial_joint_angles.items():
    joint_index = joint_names.index(joint_name)
    data.qpos[joint_index] = angle
    data.ctrl[joint_index] = angle
    #print(joint_index,angle)

# pose mocap to wrist_3
mujoco.mj_step(model, data)

#print( data.xpos[model.body('wrist_3_link').id])
data.mocap_pos[0, :] = data.xpos[model.body('wrist_3_link').id] + [ 0., -0.4, -0.1]
data.mocap_quat[:] = [[1., 0., 0., 0.]]
mujoco.mj_step(model, data)


for i in range(n_frames):
  while data.time < i/30.0:   
    data.mocap_pos[:] = data.mocap_pos + [[0.0, -0.000, -0.0001]]
    #print(data.xpos[model.body('wrist_3_link').id])
    #print(data.qpos)
    
    mujoco.mj_step(model, data)

  renderer.update_scene(data, "y")
  frame = renderer.render()
  frames.append(frame)
media.show_video(frames, fps=30)

This browser does not support the video tag.
